# Re-Ranking Using Global and Local Features
### UNIVERSITY OF TWENTE

Notebook expanding on the concept of re-ranking through the use of local features and global features along with the fusion of such features. Researchers of this notebook include:
- NIschal Neupane (n.neupane@student.utwente.nl)
- Peshmerge Morad (p.morad@student.utwente.nl)
- Aditya Poozhiyil (adityaretissinpoozhiyil@student.utwente.nl)

## Imports and Dependencies

In [163]:
import os
import sys
import json
import torch
import random
import runpy
import numpy as np
import pandas as pd
from tqdm import tqdm 

import torchvision.transforms as ttf

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances

from torchinfo import summary

from IPython.display import display

# Setup system paths relative to the current file
sys.path.insert(1, os.path.join(os.getcwd(), 'generalized_contrastive_loss', 'labeling'))
sys.path.insert(2, os.path.join(os.getcwd(), 'generalized_contrastive_loss', 'mapillary_sls'))

# Import above modules
import create_json_idx
import evaluate

# Import the other notebook file with models
%run model_notebook.ipynb

random.seed(0)
torch.manual_seed(0)
np.random.seed(0)

#### Setup GPU for Cuda

In [164]:
device = torch.device('cpu')
display_devices = False

# If cuda is available...
if torch.cuda.is_available():
    # Find GPU with most free memory and set that as the device
    mem_usage_list = [torch.cuda.mem_get_info(f'cuda:{gpu_num}')[0] for gpu_num in range(torch.cuda.device_count())]
    most_free = mem_usage_list.index(max(mem_usage_list))
    device = torch.device(f'cuda:{most_free}')
    print(f'Setting the device to {device}...\n')

    if display_devices:
        # Print GPU info on all
        for gpu_num in range(torch.cuda.device_count()):
            available_mem, total_mem = torch.cuda.mem_get_info(f'cuda:{gpu_num}')
            print(f'cuda:{gpu_num}')
            print('Memory Usage:')
            print('Total:', round(total_mem/1024**3,2), 'GB')
            print('Allocated:', round((total_mem-available_mem)/1024**3,2), 'GB')
            print('Free:   ', round(available_mem/1024**3,2), 'GB')
            print()
    # Set the default tensor type to gpu
    torch.set_default_tensor_type(torch.cuda.FloatTensor)

Setting the device to cuda:6...



#### Set the appropriate environment variables and constants

In [165]:
os.environ['MAPILLARY_ROOT'] = "/home/jovyan/APFIR/generalized_contrastive_loss/mapillary_sls/" # set the mapillary root 
os.environ['LD_LIBRARY_PATH'] = "/opt/miniconda3/lib" # Set the library path to miniconda3

In [166]:
# PATHS
DATA_ROOT = os.path.join(os.getcwd(), "generalized_contrastive_loss")

MSLS_MODELS = os.path.join(DATA_ROOT, "Models", "MSLS")
RESULTS_ROOT = os.path.join(DATA_ROOT, "results", "MSLS", "val")
PREDICTION_PATH = os.path.join(RESULTS_ROOT, "MSLS_resnext_GeM_480_GCL_predictions.txt")

MODEL_WEIGHTS = os.path.join(MSLS_MODELS, "MSLS_resnext_GeM_480_GCL.pth")

DATASET_ROOT = os.path.join(DATA_ROOT, "msls")
DATASET_TEST = os.path.join(DATASET_ROOT, "test")
DATASET_VAL = os.path.join(DATASET_ROOT, "train_val")

DATASET_VAL_SF = os.path.join(DATASET_VAL, "sf")
DATASET_VAL_CPH = os.path.join(DATASET_VAL, "cph")

In [167]:
# Indexing Parameters
RE_INDEX = False

# Model Parameters
BACKBONE = "resnext"  # backbone for the model
POOL = "GeM"  # pooling type
NORM = None  # norm type
BATCH_SIZE = 1  # batch size

DILATION = False  # Whether to add dilation layer
IMG_SIZE = (480, 640)  # Size of input image
PARAM_LENGTH = 2048  # Number of features to generate

----
----

## Indexing Dataset
Create the json indicies of the images, same as running the following command:
> python3 labeling/create_json_idx.py --dataset msls --root_dir "/home/jovyan/APFIR/generalized_contrastive_loss/msls/"

In [168]:
# Pre-check for creating indicies
def generateIndicies(replace=False):
    if not replace:
        for city_dir in os.listdir(DATASET_VAL):
            if city_dir == ".DS_Store": continue
            # get the files and check if query.json and database.json are in there
            if all(elem in os.listdir(os.path.join(DATASET_VAL, city_dir)) for elem in ['query.json', 'database.json']):
                print(f"Json index present for {city_dir}.")
                continue
            else:
                print(f"Could not find json index for {city_dir}. Creating all label indicies.")
                break
        return
    print('Indexing the data...')
    if "--dataset" not in sys.argv:
        sys.argv = [sys.argv[0]]
        sys.argv.extend(["--dataset", "msls", "--root_dir", os.path.join(os.getcwd(), DATASET_ROOT)])
    runpy.run_module('create_json_idx', run_name='__main__')
    print('Done indexing data.')


# Generates indicies if needed
generateIndicies(replace=RE_INDEX)

Json index present for sf.
Json index present for austin.
Json index present for boston.
Json index present for budapest.
Json index present for nairobi.
Json index present for cph.
Json index present for saopaulo.
Json index present for tokyo.
Json index present for helsinki.
Json index present for toronto.
Json index present for zurich.
Json index present for manila.
Json index present for london.
Json index present for phoenix.
Json index present for amsterdam.
Json index present for moscow.
Json index present for ottawa.
Json index present for amman.
Json index present for berlin.
Json index present for melbourne.
Json index present for goa.
Json index present for paris.
Json index present for trondheim.
Json index present for bangkok.


----
----

## Feature Extraction
Section regarding local and global feature extraction depending on the necessity. The extracted features are saved in results.

#### Helper Functions Defintion
All helper functions listed for feature extraction and other aspects of feature extraction.

##### Helper Function #1: Extract Features to File
These functions extract the features of the msls dataset into the respective files.

In [169]:
def extract_features_to_file(dataloader, model, f_length, feature_filepath):
    if not os.path.exists(feature_filepath):
        feats = np.zeros((len(dataloader.dataset), f_length))
        for i, batch in tqdm(enumerate(dataloader), desc="Extracting features for "+feature_filepath):
            x = model.forward(batch.cuda()).squeeze(-1).squeeze(-1)
            feats[i * dataloader.batch_size:i * dataloader.batch_size + dataloader.batch_size] = x.cpu().detach().squeeze(0)
        np.save(feature_filepath, feats)
        print(f"{feature_filepath} has been saved.")
    else:
        print(feature_filepath, "already exists. Skipping.")

##### Helper Function #2: Extract Features
These functions extract the features of the msls dataset. Depending on the necessity, there are two helper functions, one for local feature extraction and another for global feature extraction.

In [170]:
def extract_features(model, feature_type, dilation, base_filename, f_length):
    # Get all the cities in the validation set
    cities = default_cities["val"]

    # Create a transformation for each image in the dataloader
    image_transform = ttf.Compose([
        ttf.Resize(size=(IMG_SIZE[0], IMG_SIZE[1])),
        ttf.ToTensor(),
        ttf.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Loop through each city to create specific features for each
    for city in cities:
        print(city)
        query_idx = os.path.join(DATASET_ROOT, "train_val", city, "query.json")
        map_idx = os.path.join(DATASET_ROOT, "train_val", city, "database.json")

        file_savename = f"{base_filename}_{city}_dilation_{feature_type}" if dilation else f"{base_filename}_{city}_{feature_type}"

        q_dataloader = create_dataloader("test", DATASET_ROOT, query_idx, None, image_transform, BATCH_SIZE)
        q_feature_filepath = os.path.join(RESULTS_ROOT, f"{file_savename}_queryfeats.npy")
        extract_features_to_file(q_dataloader, model, f_length, q_feature_filepath)

        m_dataloader = create_dataloader("test", DATASET_ROOT, map_idx, None, image_transform, BATCH_SIZE)
        m_feature_filepath = os.path.join(RESULTS_ROOT, f"{file_savename}_mapfeats.npy")
        extract_features_to_file(m_dataloader, model, f_length, m_feature_filepath)

-------

#### Create the Model and Load Weights
Create the model from the given backend and load the weights of the model.

In [171]:
# Create the models
model = create_model(BACKBONE, POOL, norm=NORM, mode="single").to(device)

# Load the weights
try:
    model.load_state_dict(torch.load(MODEL_WEIGHTS)["model_state_dict"])
except:
    model.load_state_dict(torch.load(MODEL_WEIGHTS)["state_dict"])

if torch.cuda.is_available():
    model.cuda()

# Set model to inference mode
model.eval()
print('Model loaded!')

Using cache found in /home/jovyan/.cache/torch/hub/facebookresearch_WSL-Images_main


 the layers of the resnext101_32x8d_wsl are: odict_keys(['conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer2', 'layer3', 'layer4', 'avgpool', 'fc'])
 the layers of the resnext101_32x8d_wsl are after removing the last two layers (avgpool and fc): odict_keys(['0', '1', '2', '3', '4', '5', '6', '7'])
Number of layers: 8
0 Conv2d IS TRAINED
1 BatchNorm2d IS TRAINED
2 ReLU IS TRAINED
3 MaxPool2d IS TRAINED
4 Sequential IS TRAINED
5 Sequential IS TRAINED
6 Sequential IS TRAINED
7 Sequential IS TRAINED
Model loaded!


----

#### Generate Features
Function to generate features and create the individual models.

In [172]:
def generateFeatures(global_feat_model, input_size, hidden_dim, output_dim, feature_types, dilation, generate_summary):
    # Create the local feature model by removing the last layer and the pooling
    # local_feat_model = torch.nn.Sequential(*(list(list(global_feat_model.children())[0].children ())[:-1]), list(global_feat_model.children())[1])
    local_feat_model = torch.nn.Sequential(*(list(list(global_feat_model.children())[0].children())[:-1]))

    # global_feat_model = torch.nn.Sequential(*(list(local_feat_model.children())), list(list(global_feat_model.children())[0].children ())[-1], list(global_feat_model.children())[1])
    global_feat_model = torch.nn.Sequential(*(list(list(global_feat_model.children())[0].children())), list(global_feat_model.children())[1])

    # Create the local branch and add the layers together
    local_branch = LocalBranch(input_dim=hidden_dim, out_channel=output_dim, dilation=dilation, image_size=IMG_SIZE)
    local_branch.cuda() # move the local branch to cuda as well
    local_feat_model = torch.nn.Sequential(*(list(local_feat_model.children())), *(list(local_branch.children())))

    if not os.path.exists(RESULTS_ROOT):
        os.makedirs(RESULTS_ROOT)
    savename_base = MODEL_WEIGHTS.split("/")[-1].split(".")[0]

    if feature_types == "both":
        # generate both features
        # first, generate local features
        extract_features(
            local_feat_model,
            feature_type="local",
            dilation=dilation,
            base_filename=savename_base,
            f_length=output_dim
        )
        # then, generate global features
        extract_features(
            global_feat_model,
            feature_type="global",
            dilation=dilation,
            base_filename=savename_base,
            f_length=output_dim
        )
        if generate_summary:
            print("SUMMARY OF LOCAL FEATURE MODEL:")
            summary(local_feat_model, input_size = input_size, col_names = ("input_size", "output_size", "num_params", "trainable"), verbose = 1, depth=2)

            print("\n\nSUMMARY OF GLOBAL FEATURE MODEL:")
            summary(global_feat_model, input_size = input_size, col_names = ("input_size", "output_size", "num_params", "trainable"), verbose = 1, depth=2)
    elif feature_types == "global":
        # generate only global features
        extract_features(
            global_feat_model,
            feature_type="global",
            dilation=dilation,
            base_filename=savename_base,
            f_length=output_dim
        )
        if generate_summary:
            print("SUMMARY OF LOCAL FEATURE MODEL:")
            summary(local_feat_model, input_size = input_size, col_names = ("input_size", "output_size", "num_params", "trainable"), verbose = 1, depth=2)
        pass
    else:
        # generate only local features
        extract_features(
            local_feat_model,
            feature_type="local",
            dilation=dilation,
            base_filename=savename_base,
            f_length=output_dim
        )
        if generate_summary:
            print("SUMMARY OF LOCAL FEATURE MODEL:")
            summary(local_feat_model, input_size = input_size, col_names = ("input_size", "output_size", "num_params", "trainable"), verbose = 1, depth=2)
        pass


generateFeatures(
    model,
    input_size=(BATCH_SIZE, 3, 480, 640),
    hidden_dim=1024,
    output_dim=2048,
    feature_types="both",
    dilation=DILATION,
    generate_summary=False)

cph
/home/jovyan/APFIR/generalized_contrastive_loss/results/MSLS/val/MSLS_resnext_GeM_480_GCL_cph_local_queryfeats.npy already exists. Skipping.
/home/jovyan/APFIR/generalized_contrastive_loss/results/MSLS/val/MSLS_resnext_GeM_480_GCL_cph_local_mapfeats.npy already exists. Skipping.
sf
/home/jovyan/APFIR/generalized_contrastive_loss/results/MSLS/val/MSLS_resnext_GeM_480_GCL_sf_local_queryfeats.npy already exists. Skipping.
/home/jovyan/APFIR/generalized_contrastive_loss/results/MSLS/val/MSLS_resnext_GeM_480_GCL_sf_local_mapfeats.npy already exists. Skipping.
cph
/home/jovyan/APFIR/generalized_contrastive_loss/results/MSLS/val/MSLS_resnext_GeM_480_GCL_cph_global_queryfeats.npy already exists. Skipping.
/home/jovyan/APFIR/generalized_contrastive_loss/results/MSLS/val/MSLS_resnext_GeM_480_GCL_cph_global_mapfeats.npy already exists. Skipping.
sf
/home/jovyan/APFIR/generalized_contrastive_loss/results/MSLS/val/MSLS_resnext_GeM_480_GCL_sf_global_queryfeats.npy already exists. Skipping.
/home

----
----

## Re-Ranking
Particular section dedicated to fusion and re-ranking methods.

In [173]:
# Run the rerank notebook
%run rerank.ipynb

#### Fusion
First, we define the fusion methods for the re-ranking part.

##### Orthogonal Fusion
A method of fusion that uses vector projection to project local features onto the global features and then concatenates them.

In [174]:
def orthogonal_fusion(global_matrix,local_matrix):
    global_norm = torch.norm(global_matrix,p=2,dim=1)
    projection = torch.mm(global_matrix,local_matrix.T)
    projection = projection/(global_norm*global_norm)
    orthogonal_comp = local_matrix-projection
    fusion = torch.cat([global_matrix,orthogonal_comp],dim=1)
    return fusion

##### Modified Orthogonal Fusion
A method of fusion that uses vector projection to project local features onto the global features and then concatenates them. (Written as a nn module fully-connected layer)

In [175]:
class OrthogonalFusion(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, local_feat, global_feat):
        global_feat_norm = torch.norm(global_feat, p=2, dim=1, keepdim=True)
        projection = torch.bmm(global_feat.unsqueeze(1), torch.flatten(local_feat, start_dim=1).unsqueeze(2))
        projection = torch.bmm(global_feat.unsqueeze(2), projection).view(local_feat.size())
        projection = projection / (global_feat_norm * global_feat_norm)
        orthogonal_comp = local_feat - projection
        return torch.cat([global_feat, orthogonal_comp], dim=1)

##### Hadamard Fusion
A method of fusion that simply does a Hadamard vector multiplication.

In [176]:
def hadamard_fusion(global_features, local_features):
    """when you apply just the sequeeze() it removes all the ones
    #so (1,2048,1,1) after squeezing -> (2048), adding that extra dimension on the zero 
    #axis :- unsqueeze(0) (1,2048)"""
    if global_features.shape[0] == 1:
        x = np.multiply(local_features, global_features).squeeze().unsqueeze(0)
    else:
        # if we have batch size!=1, then we dont have to unsqueeze because
        # it wont squeeze the batch size hadamard product
        x = np.multiply(local_features, global_features).squeeze()
    return x

----

In [178]:
if DILATION:
    feat_format = MODEL_WEIGHTS.split("/")[-1].split(".")[0]+"_{city}_dilation_{feat_type}_{db}feats.npy"
    results_format = MODEL_WEIGHTS.split("/")[-1].split(".")[0]+"_dilation_results.txt"
else:
    feat_format = MODEL_WEIGHTS.split("/")[-1].split(".")[0]+"_{city}_{feat_type}_{db}feats.npy"
    results_format = MODEL_WEIGHTS.split("/")[-1].split(".")[0]+"_results.txt"
configuration = {
    "cities": ["cph", "sf"],
    "prediction_path": PREDICTION_PATH,
    "features_path": RESULTS_ROOT,
    "feat_file_format": feat_format,
    "results_file_format": results_format,
    "data_path": DATASET_VAL,
    "similarity_measure": torch.cdist,
    "fusion_method": OrthogonalFusion()
}

In [179]:
%%time
reranked_df, saved_results_file = rank(configuration)

Ranking for sf: 100%|██████████| 4525/4525 [00:02<00:00, 1551.33it/s]


Writing the results to /home/jovyan/APFIR/generalized_contrastive_loss/results/MSLS/val/MSLS_resnext_GeM_480_GCL_results.txt.
CPU times: user 55.2 s, sys: 4.15 s, total: 59.4 s
Wall time: 8.98 s


In [189]:
# if "--prediction" not in sys.argv:
#     sys.argv = [sys.argv[0]]
#     sys.argv.extend(["--prediction", RESULTS_ROOT, "--msls-root", DATASET_ROOT, "--cities", "cph,sf"])
# runpy.run_module('evaluate', run_name='__main__')

Ignoring sequence length 3 for the im2im task. (Setting to 1)
=====> cph
=====> sf


/home/jovyan/APFIR/generalized_contrastive_loss/mapillary_sls/mapillary_sls/datasets/msls.py:217: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.pIdx = np.asarray(self.pIdx)


Ignoring predictions for x3vA7Bk0HNI6rGkDpDZQUQ at line 0. It is not in the selected cities or is a panorama
Ignoring predictions for U9Vj0IV4q1psciXpj51F_w at line 1. It is not in the selected cities or is a panorama
Ignoring predictions for Eh1NwQjH4jbKcWqVJ4ZsJg at line 2. It is not in the selected cities or is a panorama
Ignoring predictions for LdiYwYkqgUfc1IYDu5ov9A at line 4. It is not in the selected cities or is a panorama
Ignoring predictions for a-OmxOaljPMY0FrFxjfuJw at line 5. It is not in the selected cities or is a panorama
Ignoring predictions for SQNDJeXa8UQ9pHht-13PNg at line 6. It is not in the selected cities or is a panorama
Ignoring predictions for GpSPPsgKpsZG7fJaoD3c1g at line 7. It is not in the selected cities or is a panorama
Ignoring predictions for 2sBLqZFw_T9vxSu_y7VJhw at line 8. It is not in the selected cities or is a panorama
Ignoring predictions for BAc687_egXNvHjMScbzgKg at line 9. It is not in the selected cities or is a panorama
Ignoring prediction

{'__name__': '__main__',
 '__file__': '/home/jovyan/APFIR/generalized_contrastive_loss/mapillary_sls/evaluate.py',
 '__cached__': '/home/jovyan/APFIR/generalized_contrastive_loss/mapillary_sls/__pycache__/evaluate.cpython-38.pyc',
 '__doc__': None,
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x7fd9ecda8ee0>,
 '__package__': '',
 '__spec__': ModuleSpec(name='evaluate', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7fd9ecda8ee0>, origin='/home/jovyan/APFIR/generalized_contrastive_loss/mapillary_sls/evaluate.py'),
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>),
  '__build_class__': <function __build_class__>,
  '__import__': <function __import__>,
  'a

In [ ]:
# from pyheat import PyHeat

In [ ]:
# ph = PyHeat("test.py")
# ph.create_heatmap()
# mytest.main(["-x","7","-y","6"]) 